# Basic examples
In this notebook we will show how to write a basic pipeline, in the **desipipe** framework. You need to have installed **desipipe** with:
```
python -m pip install git+https://github.com/cosmodesi/desipipe#egg=desipipe
```

## Task manager
Let's consider a simple example: the Monte-Carlo estimation of $\pi$.

In [1]:
import time

from desipipe import Queue, Environment, TaskManager, FileManager

# Let's instantiate a Queue, which records all tasks to be performed
# spawn = True means a manager process is spawned to distribute the tasks among workers
queue = Queue('test', base_dir='_tests', spawn=True)
# Pool of 4 workers
tm = TaskManager(queue, environ=Environment(), scheduler=dict(max_workers=4))

# We decorate the function (task) with tm.python_app
@tm.python_app
def fraction(seed=42, size=10000):
    import time
    import numpy as np
    time.sleep(5)  # wait, just to show jobs are indeed run in parallel
    x, y = np.random.uniform(-1, 1, size), np.random.uniform(-1, 1, size)
    return np.sum((x**2 + y**2) < 1.) * 1. / size


# Here we use another task manager, with only 1 worker
@tm.clone(scheduler=dict(max_workers=1)).python_app
def average(fractions):
    import numpy as np
    return np.average(fractions) * 4.


t0 = time.time()
# The following line stacks all the tasks in the queue
fractions = [fraction(seed=i) for i in range(20)]
# fractions is a list of Future instances
# We can pass them to other tasks, which creates a dependency graph
avg = average(fractions)
# At this point jobs are submitted
print('Elapsed time: {:.4f}'.format(time.time() - t0))

Elapsed time: 0.8341


In [2]:
t0 = time.time()
# result() returns the result of the function, which can take some time to complete
# in this case, ~ 20 tasks which take 5 seconds each distributed over 4 processes: typically 25 seconds
print('pi is ~ {:.4f}'.format(avg.result()))
print('Elapsed time: {:.1f}'.format(time.time() - t0))

pi is ~ 3.1439
Elapsed time: 26.9


If you re-execute the two above cells, the cached result is immediately returned.
If you modify e.g. ``fraction``, a new result will be computed. 

In [3]:
# To delete the queue:
queue.delete()